In [17]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [18]:
import torch
from torch import Tensor

## Differentiating tensors
In part 01 we looked at how tensors can be used to store data and perform vectorised operations.
In PyTorch it is possible to differentiate the results of such operations with respect to the input variables, if desired.

To do this, the dependent variable (`Tensor`) must be initialised with `requires_grad` set to `True`. Then, anytime the tensor is used in an operation the gradient of that function will be included in the result, eventually allowing it to be differentiated.

In [19]:
dep_var = torch.tensor([6.0], requires_grad=True)
dep_var

tensor([6.], requires_grad=True)

In [20]:
data = torch.randn(1)
data

tensor([-0.9007])

In [21]:
value = data*dep_var
value

tensor([-5.4044], grad_fn=<MulBackward0>)

Note that the value has a `grad_fn`, which means it can be differentiated.

`torch.autograd` contains various functions to help with this, e.g. `torch.autograd.grad`:

In [22]:
from torch.autograd import grad

In [23]:
grad(outputs=value, inputs=dep_var)

(tensor([-0.9007]),)

### Differentiating functions
Above, we performed a function on a tensor and got the result, which we then differentiated with respect to the dependent input. It is also possible to differentiate functions directly for given inputs, without getting the results of the functions. I don't particularly like this.

Note that it also computes the gradient with respect to the data, even though the data wasn't set to require gradient.

These methods are mainly designed for *functional* programming, but you'd be better off looking into the [functorch ](https://pytorch.org/functorch/stable/) extension for PyTorch, or just using [JAX](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html).

In [24]:
def func(data,dep_var):
    return data*dep_var

In [25]:
torch.autograd.functional.jacobian(func, (data,dep_var))

(tensor([[6.]]), tensor([[-0.9007]]))

In [26]:
torch.autograd.functional.hessian(func, (data,dep_var))

((tensor([[0.]]), tensor([[1.]])), (tensor([[1.]]), tensor([[0.]])))

### Multiple variables and second derivative

In [27]:
dep_var_0 = torch.tensor([6.0], requires_grad=True)
dep_var_1 = torch.tensor([-2.0], requires_grad=True)

In [28]:
data = torch.randn(1)
data

tensor([-0.3134])

In [29]:
value = (dep_var_0**data)+dep_var_1.square()
value

tensor([4.5703], grad_fn=<AddBackward0>)

To differentiate w.r.t. multiple variables, include them as a tuple in the inputs. The `retain_graph` argument allows us to to recompute the gradient, if necessary, and the `create_graph` argument makes the output also have a `grad_fn`, is applicable.

In [30]:
jac = grad(outputs=value, inputs=(dep_var_0, dep_var_1), retain_graph=0, create_graph=True)
jac

(tensor([-0.0298], grad_fn=<WhereBackward0>),
 tensor([-4.], grad_fn=<MulBackward0>))

Since the gradient has a `grad_fn`, we can compute the second derivative, too. Note that we don't get the full Hessian matrix, though, only the diagonal.

In [31]:
grad(outputs=jac, inputs=(dep_var_0, dep_var_1), retain_graph=True, create_graph=True)

(tensor([0.0065], grad_fn=<WhereBackward0>),
 tensor([2.], grad_fn=<MulBackward0>))

#### Full Hessian

If you know that you'll be later computing the Hessian, or even just Jacobians, I find it best to have all the dependent variables in a single Tensor:

In [32]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [33]:
value = (dep_vars[0]**data)+dep_vars[1].square()
value

tensor([4.5703], grad_fn=<AddBackward0>)

We compute the Jacobian as normal. This now returns the Jacobian in a single tensor, rather than a tuple of tensors.

In [34]:
jac = grad(outputs=value, inputs=dep_vars, retain_graph=True, create_graph=True)
jac

(tensor([-0.0298, -4.0000], grad_fn=<AddBackward0>),)

Now we try to compute the Hessian:

In [35]:
grad(outputs=jac, inputs=dep_vars, retain_graph=True, create_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs

Oh dear! The output needs to be a single value, and we provided two values. Instead we can supply each value in turn:

In [36]:
grad(outputs=jac[0][0], inputs=dep_vars, retain_graph=True, create_graph=True), grad(outputs=jac[0][1], inputs=dep_vars, retain_graph=True, create_graph=True)

((tensor([0.0065, 0.0000], grad_fn=<AddBackward0>),),
 (tensor([0., 2.], grad_fn=<AddBackward0>),))

So now we get the full Hessian matrix. (In this case the off-diagonals were zero, but they might not always be)

## Batched gradients and better Hessians
We already saw that the `grad` function has trouble dealing with non-scalar outputs, which meant we needed to call it twice and get a tuple. A more general way to do this, would be to iterate over each element of the Jacobian and stack the Hessian rows into a tensor:

In [37]:
torch.stack([grad(outputs=j, inputs=dep_vars, retain_graph=True, create_graph=True)[0] for j in jac[0].unbind(0)])  # unbind alows us to iterate through the tensor along the specified dimension

tensor([[0.0065, 0.0000],
        [0.0000, 2.0000]], grad_fn=<StackBackward0>)

In [38]:
%%timeit
torch.stack([grad(outputs=j, inputs=dep_vars, retain_graph=True, create_graph=True)[0] for j in jac[0].unbind(0)])  # unbind alows us to iterate through the tensor along the specified dimension

282 µs ± 5.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


A slightly quicker way is to still feed in the full Jacobian, but use the `grad_outputs` to *switch on* which element we want to differentiate. `torch.eye` creates an identity matrix of a given size, and iterating through it will provide one-hot vectors.

In [39]:
torch.eye(len(jac[0]))

tensor([[1., 0.],
        [0., 1.]])

In [40]:
torch.stack([grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0] for i in torch.eye(len(jac[0])).unbind(0)])

tensor([[0.0065, 0.0000],
        [0.0000, 2.0000]], grad_fn=<StackBackward0>)

In [41]:
%%timeit 
torch.stack([grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0] for i in torch.eye(len(jac[0])).unbind(0)])

263 µs ± 5.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Vectorised method
The above method works, but still relies on a python for-loop to provide serial calls to `grad`. It would be better to instead perform all calls in parallel. We can do this using the PyTorch vmap function, although it is still experimental.

`vmap` takes a function and a set of input arguments and will implicitly compute the function values by unbinding the inputs, and will then stack the results to a tensor:

In [42]:
from torch._vmap_internals import _vmap as vmap

In [43]:
vmap(lambda i: grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0])(torch.eye(len(jac[0])))

tensor([[0.0065, 0.0000],
        [0.0000, 2.0000]], grad_fn=<AddBackward0>)

In [44]:
%%timeit
vmap(lambda i: grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0])(torch.eye(len(jac[0])))

234 µs ± 8.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Batched gradients
The "non-scalar output" issue doesn't just apply to Hessians, what if we want to efficiently differentiate a batch of items independently?

In [45]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [46]:
data = torch.rand(10,2)

In [47]:
values = (dep_vars[0]**data)+dep_vars[1].square()
values

tensor([[8.6426, 5.9181],
        [5.2748, 5.0024],
        [6.4629, 5.8897],
        [6.8285, 8.4848],
        [6.1103, 5.5370],
        [7.8879, 9.5620],
        [7.9692, 7.0618],
        [6.0504, 5.1230],
        [5.2790, 5.2493],
        [7.8670, 9.2011]], grad_fn=<AddBackward0>)

In [48]:
grad(outputs=values, inputs=dep_vars, retain_graph=True, create_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs

The trick is to reuse our vmap'd Jacobian, however even if we iterate over each row of the output values, it still isn't a scalar value; we need iterate over each element.

Rather than having the iteration adapt to every possible output shape, it is instead more convenient to write a generalised Jacobian function that works for any shape, by flattening and reshaping the inputs.

In [49]:
def jacobian(y: Tensor, x: Tensor, create_graph: bool = False, allow_unused: bool = True) -> Tensor:
    r"""
    Computes the Jacobian (dy/dx) of y with respect to variables x. x and y can have multiple elements.
    If y has multiple elements then computation is vectorised via vmap.

    Arguments:
        y: tensor to be differentiated
        x: dependent variables
        create_graph: If True, graph of the derivative will
            be constructed, allowing to compute higher order derivative products.
            Default: False.
        allow_unused: If False, specifying inputs that were not
            used when computing outputs (and therefore their grad is always

    Returns:
        dy/dx tensor of shape y.shape+x.shape
    """

    if len(y) == 0:
        return None
    flat_y = y.reshape(-1)

    def get_vjp(v: Tensor) -> Tensor:
        return torch.autograd.grad(flat_y, x, v, retain_graph=True, create_graph=create_graph, allow_unused=allow_unused)[0].reshape(x.shape)

    return vmap(get_vjp)(torch.eye(len(flat_y), device=y.device)).reshape(y.shape + x.shape)

In [50]:
jacobian(values, dep_vars)

tensor([[[ 6.6300e-01, -4.0000e+00],
         [ 1.1620e-01, -4.0000e+00]],

        [[ 2.8785e-02, -4.0000e+00],
         [ 2.2137e-04, -4.0000e+00]],

        [[ 2.0650e-01, -4.0000e+00],
         [ 1.1186e-01, -4.0000e+00]],

        [[ 2.7357e-01, -4.0000e+00],
         [ 6.2604e-01, -4.0000e+00]],

        [[ 1.4660e-01, -4.0000e+00],
         [ 6.1452e-02, -4.0000e+00]],

        [[ 4.9106e-01, -4.0000e+00],
         [ 8.8779e-01, -4.0000e+00]],

        [[ 5.0898e-01, -4.0000e+00],
         [ 3.1869e-01, -4.0000e+00]],

        [[ 1.3695e-01, -4.0000e+00],
         [ 1.2120e-02, -4.0000e+00]],

        [[ 2.9275e-02, -4.0000e+00],
         [ 2.5864e-02, -4.0000e+00]],

        [[ 4.8649e-01, -4.0000e+00],
         [ 7.9771e-01, -4.0000e+00]]])

In [51]:
jacobian(values, dep_vars).shape

torch.Size([10, 2, 2])

In [53]:
values.shape, dep_vars.shape

(torch.Size([10, 2]), torch.Size([2]))

This gives us the Jacobian of every element

## No-grad contexts
Calculations performed involving tensors with gradient incur an increased cost in terms of time and memory. In cases where gradient tracking isn't required, the context manager `no_grad` may be used:

In [168]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [176]:
data = torch.rand(10,2)

In [187]:
%%timeit
(dep_vars[0]**data)+dep_vars[1].square()

14.3 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [191]:
%%timeit
with torch.no_grad():
    (dep_vars[0]**data)+dep_vars[1].square()

10.2 µs ± 47.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


The `inference_mode` context is even more performant:

In [192]:
%%timeit
with torch.inference_mode():
    (dep_vars[0]**data)+dep_vars[1].square()

9.71 µs ± 59.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Modifying tensor with gradient
Once a tensor is set to have gradient, in-place modifications to it will result in an exception.

In [3]:
a = Tensor([3.0])

In [4]:
a[0] = 4.0

In [6]:
a.requires_grad = True

In [8]:
a[0] = 1.0

RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

Here, the `no_grad` context can be used

In [12]:
with torch.no_grad():
    a[0] = 1.0
a

tensor([1.], requires_grad=True)

Or its `data` can be modified directly

In [16]:
a.data[0] = 7.0
a

tensor([7.], requires_grad=True)